# Organised Teams Statistics

Analysis of OpenStreetMap contributions from organised teams (including corporations).

In [1]:
import duckdb
import util

util.init()

## Total and Percent of Edits and Contributors from Organised Teams Per Month

In [2]:
# Organised teams statistics per month
df = duckdb.sql("""
WITH monthly_total AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        COUNT(DISTINCT user_name) as total_contributors,
        CAST(SUM(edit_count) as BIGINT) as total_edits
    FROM '../changeset_data/year=*/month=*/*.parquet'
    GROUP BY year, month
),
monthly_organised_team AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        COUNT(DISTINCT user_name) as team_contributors,
        CAST(SUM(edit_count) as BIGINT) as team_edits
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE organised_team IS NOT NULL
    GROUP BY year, month
)
SELECT 
    mt.months,
    mt.total_contributors as "Total Contributors",
    COALESCE(mot.team_contributors, 0) as "Organised Team Contributors",
    mt.total_edits as "Total Edits",
    COALESCE(mot.team_edits, 0) as "Organised Team Edits",
    ROUND((COALESCE(mot.team_contributors, 0) * 100.0) / mt.total_contributors, 2) as "Percent Contributors from Organised Teams",
    ROUND((COALESCE(mot.team_edits, 0) * 100.0) / mt.total_edits, 2) as "Percent Edits from Organised Teams"
FROM monthly_total mt
LEFT JOIN monthly_organised_team mot ON mt.year = mot.year AND mt.month = mot.month
ORDER BY mt.year, mt.month
""").df()

util.show_figure(
    [
        util.FigureConfig(
            title="Percentage of Edits from Organised Teams",
            label="Edits Percentage",
            x_col="months",
            y_col="Percent Edits from Organised Teams",
            y_unit_hover_template="%",
            query_or_df=df,
        ),
        util.FigureConfig(
            title="Monthly Edits from Organised Teams",
            label="Edits",
            x_col="months",
            y_col="Organised Team Edits",
            query_or_df=df,
        ),
        util.FigureConfig(
            title="Percentage of Contributors from Organised Teams",
            label="Contributors Percentage",
            x_col="months",
            y_col="Percent Contributors from Organised Teams",
            y_unit_hover_template="%",
            query_or_df=df,
        ),
        util.FigureConfig(
            title="Monthly Contributors from Organised Teams",
            label="Contributors",
            x_col="months",
            y_col="Organised Team Contributors",
            query_or_df=df,
        ),
    ]
)

## Monthly Contributors, New Contributors, and Edits Per Organised Team (Top 10 Plot)

In [3]:
# Top 10 organised teams by contributors
df_top10 = duckdb.sql("""
WITH top_organised_teams AS (
    SELECT organised_team
    FROM (
        SELECT
            organised_team,
            SUM(edit_count) as total_edits
        FROM '../changeset_data/year=*/month=*/*.parquet'
        WHERE organised_team IS NOT NULL
        GROUP BY organised_team
        ORDER BY total_edits DESC
        LIMIT 10
    )
),
user_first_appearance AS (
    SELECT
        user_name,
        year,
        month,
        organised_team,
        ROW_NUMBER() OVER (PARTITION BY user_name ORDER BY year, month) as rn
    FROM (
        SELECT DISTINCT user_name, year, month, organised_team
        FROM '../changeset_data/year=*/month=*/*.parquet'
        WHERE organised_team IN (SELECT organised_team FROM top_organised_teams)
    )
),
first_appearances AS (
    SELECT user_name, year, month, organised_team
    FROM user_first_appearance
    WHERE rn = 1
),
monthly_contributors AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        organised_team,
        COUNT(DISTINCT user_name) as "Contributors",
        CAST(SUM(edit_count) as BIGINT) as "Edits"
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE organised_team IN (SELECT organised_team FROM top_organised_teams)
    GROUP BY year, month, organised_team
),
monthly_new_contributors AS (
    SELECT
        year,
        month,
        organised_team,
        COUNT(DISTINCT user_name) as "New Contributors"
    FROM first_appearances
    GROUP BY year, month, organised_team
)
SELECT 
    mc.months,
    mc.organised_team,
    mc."Contributors",
    COALESCE(mnc."New Contributors", 0) as "New Contributors",
    mc."Edits"
FROM monthly_contributors mc
LEFT JOIN monthly_new_contributors mnc ON mc.year = mnc.year AND mc.month = mnc.month AND mc.organised_team = mnc.organised_team
ORDER BY mc.year, mc.month, mc.organised_team
""").df()

util.show_figure(
    [
        util.FigureConfig(
            title="Monthly Edits by Top 10 Organised Teams",
            label="Edits",
            x_col="months",
            y_col="Edits",
            group_col="organised_team",
            query_or_df=df_top10,
        ),
        util.FigureConfig(
            title="Monthly Contributors by Top 10 Organised Teams",
            label="Contributors",
            x_col="months",
            y_col="Contributors",
            group_col="organised_team",
            query_or_df=df_top10,
        ),
        util.FigureConfig(
            title="Monthly New Contributors by Top 10 Organised Teams",
            label="New Contributors",
            x_col="months",
            y_col="New Contributors",
            group_col="organised_team",
            query_or_df=df_top10,
        ),
    ]
)

## All Organised Teams Table

In [4]:
query = """
WITH organised_team_totals AS (
    SELECT
        organised_team as "Organised Team",
        CAST(SUM(edit_count) as BIGINT) as total_edits_all_time,
        CAST(COUNT(DISTINCT user_name) as BIGINT) as total_contributors_all_time
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE organised_team IS NOT NULL
    GROUP BY organised_team
),
yearly_metrics AS (
    SELECT
        d.year,
        d.organised_team as "Organised Team",
        CAST(SUM(d.edit_count) as BIGINT) as "Edits",
        CAST(COUNT(DISTINCT d.user_name) as BIGINT) as "Contributors"
    FROM '../changeset_data/year=*/month=*/*.parquet' d
    WHERE d.organised_team IS NOT NULL
    GROUP BY d.year, d.organised_team
)
SELECT 
    ym.year,
    ym."Organised Team",
    ym."Edits", 
    ym."Contributors",
    ott.total_edits_all_time as "Total Edits",
    ott.total_contributors_all_time as "Total Contributors"
FROM yearly_metrics ym
JOIN organised_team_totals ott ON ym."Organised Team" = ott."Organised Team"
ORDER BY year DESC, "Edits" DESC
"""
df_all = duckdb.sql(query).df()

table_configs = [
    util.TableConfig(
        title="Edits Per Organised Team by Year",
        label="Edits",
        query_or_df=df_all,
        x_axis_col="year",
        y_axis_col="Organised Team",
        value_col="Edits",
        center_columns=["Rank", "Organised Team"],
        sum_col="Total Edits",
    ),
    util.TableConfig(
        title="Contributors Per Organised Team by Year",
        label="Contributors",
        query_or_df=df_all,
        x_axis_col="year",
        y_axis_col="Organised Team",
        value_col="Contributors",
        center_columns=["Rank", "Organised Team"],
        sum_col="Total Contributors",
    ),
]

util.show_tables(table_configs)

Rank,Organised Team,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,Total Edits
1,Meta,0,0,0,0,0,0,0,0,0,"78,815","4,296,035","20,247,234","28,957,510","50,673,799","83,060,907","51,973,945","25,494,310","1,074,076","1,173,709","267,030,340"
2,Kaart,0,"22,922","7,559","364,177","976,391","730,863","780,037","623,276","1,509,967","6,907,406","7,390,957","8,620,818","21,691,873","28,594,458","19,554,990","17,457,242","14,491,270","10,149,722","7,230,335","147,104,263"
3,Apple,0,0,0,0,0,0,0,0,0,0,"601,578","3,974,945","13,770,839","17,215,025","8,878,281","13,050,921","10,308,898","5,385,095","1,691,394","74,876,976"
4,Amazon,0,0,0,0,0,0,0,0,0,0,68,"1,171,512","15,006,820","29,407,768","19,940,677","1,456,200","1,034,876","76,744","98,718","68,193,383"
5,Mapbox,0,0,"45,339","130,377","336,422","1,812,554","2,975,135","5,001,072","10,364,605","7,176,557","6,020,903","5,478,908","1,058,384","1,549,755","1,151,670","1,848,191","1,427,806","1,787,103","3,412,546","51,577,327"
6,Microsoft,0,0,846,"7,795",0,184,0,0,0,987,"44,465","3,308,373","9,463,749","6,588,507","7,790,014","2,157,361","2,245,106","933,667","2,523,502","35,064,556"
7,DigitalEgypt,0,0,0,0,0,0,0,0,0,0,0,"7,815","4,835,241","4,385,173","3,475,331","4,657,479","4,054,066","3,732,908","1,728,077","26,876,090"
8,TomTom,0,0,0,0,0,0,0,0,0,0,"13,356",904,14,256,"5,163,115","5,383,053","5,994,057","5,209,686","3,698,322","25,462,763"
9,Grab,0,0,0,0,0,0,61,10,"360,575","158,382","1,122,025","847,285","3,077,576","2,958,667","8,916,251","3,993,110","538,887","377,096","579,566","22,929,491"
10,GeoCompas,0,0,0,0,"23,957","1,421,771","2,478,856","2,860,814","3,100,322","1,768,042","582,325","1,756,308","580,203","456,227","99,606","844,325","967,220","730,673","848,040","18,518,689"
